El presente notebook contiene los procesos y métodos llevados a cabo para hacer un análisis descriptivo de las variables. Las conclusiones de esto se pueden ver en la presentación que se adjunta.

In [219]:
import pandas as pd
import numpy as np
from os import path, getcwd
from typing import Dict


In [220]:
df= pd.read_excel('./data/Caso 01 - Cobranza Sector Financiero.xlsx')

In [221]:
import findspark
findspark.init()

In [222]:
from pyspark.sql import SparkSession
from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [223]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DoubleType , LongType, DateType
from pyspark.sql.functions import col, desc, asc, isnan, when, count, avg, min, max, variance, percentile_approx, stddev

In [224]:
spark = SparkSession.builder.appName("PysparkSession")\
                            .config("spark.shuffle.sql.partitions", 5)\
                            .getOrCreate()

In [225]:
spark

In [226]:
file_name='Caso 01 - Cobranza Sector Financiero.xlsx'

In [227]:
file_base_name: str = file_name.rsplit('.', maxsplit=1)[0]

In [228]:
files_path: str = path.join(getcwd(), 'data')

In [229]:
files_path

'C:\\Users\\LENOVO\\hackathon\\data'

In [230]:
cols_schemas: Dict[str, dict] = {
    "Consecutivo Documento Deudor": {
        "new_name": "consecutivo_documento_deudor",
        "type": IntegerType(),
        "nullable": False
    },
    "Nit": {
        "new_name": "nit",
        "type": IntegerType(),
        "nullable": False
    },
    "Nombres": {
        "new_name": "nombres",
        "type": StringType(),
        "nullable": True
    },
    "Numero Documento": {
        "new_name": "numero_documento",
        "type": IntegerType(),
        "nullable": True
    },
    "Tipo De Producto": {
        "new_name": "tipo_producto",
        "type": IntegerType(),
        "nullable": True
    },
    "PRODUCTO": {
        "new_name": "producto",
        "type": StringType(),
        "nullable": False
    },
    "Valor Obligacion": {
        "new_name": "valor_obligacion",
        "type": DoubleType(),
        "nullable": False
    },
    "Valor Vencido": {
        "new_name": "valor_vencido",
        "type": DoubleType(),
        "nullable": False
    },
    "Regional": {
        "new_name": "regional",
        "type": IntegerType(),
        "nullable": False
    },
    "REGION": {
        "new_name": "region",
        "type": StringType(),
        "nullable": False
    },
    "Abogado": {
        "new_name": "abogado",
        "type": IntegerType(),
        "nullable": False
    },
    "Dias Mora": {
        "new_name": "dias_mora",
        "type": IntegerType(),
        "nullable": False
    },
    "FRANJA": {
        "new_name": "franja",
        "type": StringType(),
        "nullable": False
    },
    "Fecha Traslado Para Cobro": {
        "new_name": "fecha_traslado_cobro",
        "type": DateType(),
        "nullable": False
    },
    "Fecha Ultima Facturacion": {
        "new_name": "fecha_ultima_facturacion",
        "type": DateType(),
        "nullable": False
    },
    "Segmento": {
        "new_name": "segmento",
        "type": StringType(),
        "nullable": False
    },
    "Oficina Radicacion": {
        "new_name": "oficina_radicacion",
        "type": IntegerType(),
        "nullable": False
    },
    "Red": {
        "new_name": "red",
        "type": IntegerType(),
        "nullable": False
    },
    "Cuadrante": {
        "new_name": "cuadrante",
        "type": StringType(),
        "nullable": False
    },
    "Descripcion Causal": {
        "new_name": "descripcion_causal",
        "type": StringType(),
        "nullable": False
    },
    "Calificacion Real": {
        "new_name": "calificacion_real",
        "type": IntegerType(),
        "nullable": False
    },
    "Fecha De Perfeccionamiento": {
        "new_name": "fecha_Perfeccionamiento",
        "type": DateType(),
        "nullable": False
    },
    "Endeudamiento": {
        "new_name": "endeudamiento",
        "type": FloatType(),
        "nullable": False
    },
    "Rango endeudamiento": {
        "new_name": "rango_endeudamiento",
        "type": StringType(),
        "nullable": False
    },
    "Sector Economico": {
        "new_name": "sector_economico",
        "type": StringType(),
        "nullable": False
    },
    "Profesion": {
        "new_name": "profesion",
        "type": StringType(),
        "nullable": False
    },
    "Ocupacion": {
        "new_name": "ocupacion",
        "type": StringType(),
        "nullable": False
    },
}

In [231]:
df: pd.DataFrame = pd.read_excel(path.join(files_path, file_name), engine='openpyxl')

In [232]:
csv_path = path.join(files_path, file_base_name + ".csv")

In [233]:
df.to_csv(csv_path, index=False)

In [234]:
df.columns

Index(['Consecutivo Documento Deudor', 'Nit', 'Nombres', 'Numero Documento',
       'Tipo De Producto', 'PRODUCTO', 'Valor Obligacion', 'Valor Vencido',
       'Regional', 'REGION', 'Abogado', 'Dias Mora', 'FRANJA',
       'Fecha Traslado Para Cobro', 'Fecha Ultima Facturacion', 'Segmento',
       'Oficina Radicacion', 'Red', 'Cuadrante', 'Descripcion Causal',
       'Calificacion Real', 'Fecha De Perfeccionamiento', 'Endeudamiento',
       'Rango endeudamiento', 'Sector Economico', 'Profesion', 'Ocupacion'],
      dtype='object')

In [235]:
del df

In [236]:
def schema() -> StructType:
    return StructType([
        StructField(cols_schemas[key]['new_name'], 
                    cols_schemas[key]["type"], 
                    cols_schemas[key]["nullable"]) 
        for key in cols_schemas.keys()
    ])

In [237]:
df_spark = spark.read.format('csv')\
                     .options(header='True') \
                     .option('delimiter', ',') \
                     .schema(schema())\
                     .csv(csv_path)

In [238]:
df_spark.printSchema()

root
 |-- consecutivo_documento_deudor: integer (nullable = true)
 |-- nit: integer (nullable = true)
 |-- nombres: string (nullable = true)
 |-- numero_documento: integer (nullable = true)
 |-- tipo_producto: integer (nullable = true)
 |-- producto: string (nullable = true)
 |-- valor_obligacion: double (nullable = true)
 |-- valor_vencido: double (nullable = true)
 |-- regional: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- abogado: integer (nullable = true)
 |-- dias_mora: integer (nullable = true)
 |-- franja: string (nullable = true)
 |-- fecha_traslado_cobro: date (nullable = true)
 |-- fecha_ultima_facturacion: date (nullable = true)
 |-- segmento: string (nullable = true)
 |-- oficina_radicacion: integer (nullable = true)
 |-- red: integer (nullable = true)
 |-- cuadrante: string (nullable = true)
 |-- descripcion_causal: string (nullable = true)
 |-- calificacion_real: integer (nullable = true)
 |-- fecha_Perfeccionamiento: date (nullable = true)
 |-- en

In [239]:
df_spark.show()

+----------------------------+---+-------+----------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+------+--------------------+------------------------+-------------+------------------+---+--------------------+--------------------+-----------------+-----------------------+-------------+-------------------+--------------------+-------------+-------------+
|consecutivo_documento_deudor|nit|nombres|numero_documento|tipo_producto|           producto|valor_obligacion|valor_vencido|regional|   region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|     segmento|oficina_radicacion|red|           cuadrante|  descripcion_causal|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|    sector_economico|    profesion|    ocupacion|
+----------------------------+---+-------+----------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+-

In [240]:
from pyspark.sql.functions import col, desc, asc, isnan, when, count, avg, min, max

df_spark.select([
    count(when(col(column).isNull(), column)).alias(column) for column in df_spark.columns   
]).show()

+----------------------------+---+-------+----------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+------------------+-----------------+-----------------------+-------------+-------------------+----------------+---------+---------+
|consecutivo_documento_deudor|nit|nombres|numero_documento|tipo_producto|producto|valor_obligacion|valor_vencido|regional|region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|segmento|oficina_radicacion|red|cuadrante|descripcion_causal|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|sector_economico|profesion|ocupacion|
+----------------------------+---+-------+----------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+--------

In [241]:
# drop_cols = ['consecutivo_documento_deudor', 'regional']

In [242]:
# for col in drop_cols:
#     df_spark = df_spark.drop(col)

### Medidas de tendencia central

In [243]:
def median_pyspark(df_spark):
    return {
        'median_' + cols_schemas[col]['new_name']: df_spark.select(
            percentile_approx(cols_schemas[col]['new_name'], 0.5).alias('median_'+cols_schemas[col]['new_name'])
        ).collect()[0][0] if cols_schemas[col]['new_name'] in df_spark.columns else None
        for col in cols_schemas.keys()
    }
        

In [244]:
def mean_pyspark(df_spark):
    not_useful = ['fecha_traslado_cobro', 'fecha_ultima_facturacion', 'fecha_Perfeccionamiento']
    return {
        'avg_' + cols_schemas[col]['new_name']: df_spark.select(
            avg(cols_schemas[col]['new_name']).alias('avg_'+cols_schemas[col]['new_name'])
        ).collect()[0][0] if cols_schemas[col]['new_name'] in df_spark.columns and cols_schemas[col]['new_name'] not in not_useful else None
        for col in cols_schemas.keys()
    }  

In [245]:
def mode_pyspark(df_spark):
    dic_moda = {}

    list_dic = []

    for item,_ in df_spark.dtypes:

        try:
            key , value = item+'_Moda', df_spark.filter(col(item).isNotNull()).groupBy(item)\
                                                                              .count()\
                                                                              .orderBy("count", ascending=False)\
                                                                              .collect()[0]\
                                                                              .__getitem__(item)
            dic_moda[key] = value
        except:
            pass

    return dic_moda

In [265]:
def variance_pyspark(df_spark) -> dict:
    not_useful = ['fecha_traslado_cobro', 'fecha_ultima_facturacion', 'fecha_Perfeccionamiento']
    return {
        'var_' + cols_schemas[col]['new_name']: df_spark.select(
            variance(cols_schemas[col]['new_name']).alias('var_'+cols_schemas[col]['new_name'])
        ).collect()[0][0] if cols_schemas[col]['new_name'] in df_spark.columns and cols_schemas[col]['new_name'] not in not_useful else None
        for col in cols_schemas.keys()
    }

In [258]:
def stddev_pyspark(df_spark) -> dict:
    not_useful = ['fecha_traslado_cobro', 'fecha_ultima_facturacion', 'fecha_Perfeccionamiento']
    
    return {
        'std_' + cols_schemas[col]['new_name']: df_spark.select(
            stddev(cols_schemas[col]['new_name']).alias('std_'+cols_schemas[col]['new_name'])
        ).collect()[0][0] if cols_schemas[col]['new_name'] in df_spark.columns and cols_schemas[col]['new_name'] not in not_useful else None
        for col in cols_schemas.keys()
    }

In [248]:
def coef_var_pyspark(df_spark) -> dict:
        variables_numericas = [c for c,t in df_spark.dtypes if t in ['bigint', 'int', 'float']]
        cv = [(stddev(col)/avg(col)).alias('cv_' + col) for col in variables_numericas]
        results = df_spark.select(cv).first()
        coef_var_dict = dict()

        for col in variables_numericas:
            cvi = results['cv_'+ col]
            coef_var_dict[col] = cvi
        return coef_var_dict

In [249]:
df_spark

DataFrame[consecutivo_documento_deudor: int, nit: int, nombres: string, numero_documento: int, tipo_producto: int, producto: string, valor_obligacion: double, valor_vencido: double, regional: int, region: string, abogado: int, dias_mora: int, franja: string, fecha_traslado_cobro: date, fecha_ultima_facturacion: date, segmento: string, oficina_radicacion: int, red: int, cuadrante: string, descripcion_causal: string, calificacion_real: int, fecha_Perfeccionamiento: date, endeudamiento: float, rango_endeudamiento: string, sector_economico: string, profesion: string, ocupacion: string]

In [250]:
median_pyspark(df_spark)

{'median_consecutivo_documento_deudor': 25785404,
 'median_nit': 563943,
 'median_nombres': None,
 'median_numero_documento': None,
 'median_tipo_producto': 3,
 'median_producto': None,
 'median_valor_obligacion': 1926412.0,
 'median_valor_vencido': 241898.0,
 'median_regional': 2099,
 'median_region': None,
 'median_abogado': 109978,
 'median_dias_mora': 30,
 'median_franja': None,
 'median_fecha_traslado_cobro': datetime.date(2015, 6, 21),
 'median_fecha_ultima_facturacion': datetime.date(2015, 6, 25),
 'median_segmento': None,
 'median_oficina_radicacion': 426,
 'median_red': 1,
 'median_cuadrante': None,
 'median_descripcion_causal': None,
 'median_calificacion_real': 2,
 'median_fecha_Perfeccionamiento': datetime.date(2013, 8, 2),
 'median_endeudamiento': 4598956.0,
 'median_rango_endeudamiento': None,
 'median_sector_economico': None,
 'median_profesion': None,
 'median_ocupacion': None}

In [251]:
mean_pyspark(df_spark)

{'avg_consecutivo_documento_deudor': 23155282.778928142,
 'avg_nit': 485493.68977108476,
 'avg_nombres': None,
 'avg_numero_documento': None,
 'avg_tipo_producto': 11.400696942194136,
 'avg_producto': None,
 'avg_valor_obligacion': 7069209.776261843,
 'avg_valor_vencido': 706280.6968713797,
 'avg_regional': 3589.738725111208,
 'avg_region': None,
 'avg_abogado': 109977.5451051576,
 'avg_dias_mora': 47.099319865093506,
 'avg_franja': None,
 'avg_fecha_traslado_cobro': None,
 'avg_fecha_ultima_facturacion': None,
 'avg_segmento': None,
 'avg_oficina_radicacion': 443.38626508454064,
 'avg_red': 1.2986879110782434,
 'avg_cuadrante': None,
 'avg_descripcion_causal': None,
 'avg_calificacion_real': 1.7216601118244872,
 'avg_fecha_Perfeccionamiento': None,
 'avg_endeudamiento': 14320682.09399108,
 'avg_rango_endeudamiento': None,
 'avg_sector_economico': None,
 'avg_profesion': None,
 'avg_ocupacion': None}

In [252]:
mode_pyspark(df_spark)

{'consecutivo_documento_deudor_Moda': 19927288,
 'nit_Moda': 246428,
 'tipo_producto_Moda': 3,
 'producto_Moda': 'TARJETAS DE CREDITO',
 'valor_obligacion_Moda': 46869.0,
 'valor_vencido_Moda': 21274.0,
 'regional_Moda': 2099,
 'region_Moda': 'BOGOTA',
 'abogado_Moda': 109978,
 'dias_mora_Moda': 30,
 'franja_Moda': '<30',
 'fecha_traslado_cobro_Moda': datetime.date(2015, 6, 21),
 'fecha_ultima_facturacion_Moda': datetime.date(2015, 6, 26),
 'segmento_Moda': 'PERSONAL PLUS',
 'oficina_radicacion_Moda': 999,
 'red_Moda': 1,
 'cuadrante_Moda': 'TIENE VOLUNTAD Y TIENE CAPACIDAD REDUCIDA',
 'descripcion_causal_Moda': 'DESCUIDO',
 'calificacion_real_Moda': 2,
 'fecha_Perfeccionamiento_Moda': datetime.date(2015, 6, 12),
 'endeudamiento_Moda': 29422040.0,
 'rango_endeudamiento_Moda': '2015-01-01 00:00:00',
 'sector_economico_Moda': 'SERVICIOS SOCIALES Y DE SALUD',
 'profesion_Moda': 'SIN PROFESION',
 'ocupacion_Moda': 'SIN OFICIO'}

In [266]:
variance_pyspark(df_spark)

{'var_consecutivo_documento_deudor': 55936900723296.28,
 'var_nit': 73678321193.9615,
 'var_nombres': None,
 'var_numero_documento': None,
 'var_tipo_producto': 549.0277399171964,
 'var_producto': None,
 'var_valor_obligacion': 230740311094255.62,
 'var_valor_vencido': 3529358596766.2466,
 'var_regional': 5393963.712385901,
 'var_region': None,
 'var_abogado': 1.6870916252927441,
 'var_dias_mora': 4914.784636047714,
 'var_franja': None,
 'var_fecha_traslado_cobro': None,
 'var_fecha_ultima_facturacion': None,
 'var_segmento': None,
 'var_oficina_radicacion': 91496.6734115112,
 'var_red': 0.4089020948209133,
 'var_cuadrante': None,
 'var_descripcion_causal': None,
 'var_calificacion_real': 0.8498168975397471,
 'var_fecha_Perfeccionamiento': None,
 'var_endeudamiento': 850730475757888.0,
 'var_rango_endeudamiento': None,
 'var_sector_economico': None,
 'var_profesion': None,
 'var_ocupacion': None}

In [263]:
stddev_pyspark(df_spark)

{'std_consecutivo_documento_deudor': 7479097.587496521,
 'std_nit': 271437.5088191783,
 'std_nombres': None,
 'std_numero_documento': None,
 'std_tipo_producto': 23.431340975650464,
 'std_producto': None,
 'std_valor_obligacion': 15190138.613398353,
 'std_valor_vencido': 1878658.7228036514,
 'std_regional': 2322.4908422609337,
 'std_region': None,
 'std_abogado': 1.2988809126678027,
 'std_dias_mora': 70.1055250037236,
 'std_franja': None,
 'std_fecha_traslado_cobro': None,
 'std_fecha_ultima_facturacion': None,
 'std_segmento': None,
 'std_oficina_radicacion': 302.4841705139481,
 'std_red': 0.6394545291269061,
 'std_cuadrante': None,
 'std_descripcion_causal': None,
 'std_calificacion_real': 0.9218551391296504,
 'std_fecha_Perfeccionamiento': None,
 'std_endeudamiento': 29167284.339785356,
 'std_rango_endeudamiento': None,
 'std_sector_economico': None,
 'std_profesion': None,
 'std_ocupacion': None}

In [260]:
coef_var_pyspark(df_spark)

{'consecutivo_documento_deudor': 0.3229974627778106,
 'nit': 0.5590958534335717,
 'numero_documento': None,
 'tipo_producto': 2.0552551387389966,
 'regional': 0.6469804685267126,
 'abogado': 1.1810419221722465e-05,
 'dias_mora': 1.4884615150394258,
 'oficina_radicacion': 0.6822136686085062,
 'red': 0.49238506316424796,
 'calificacion_real': 0.5354454882228392,
 'endeudamiento': 2.036724518312146}

### Endeudamiento

In [270]:
minimo= df_spark.select(min('endeudamiento').alias('minimo valor endeudado')).collect()[0][0]

In [271]:
minimo

18044.0

In [275]:
maximo= df_spark.select(max('endeudamiento').alias('maximo valor endeudado')).collect()[0][0]

In [276]:
maximo

2012549248.0

In [287]:
df_region_deuda=df_spark.groupBy('region').avg('endeudamiento').show()

+---------+--------------------+
|   region|  avg(endeudamiento)|
+---------+--------------------+
|ANTIOQUIA|1.2605151593091443E7|
|   CARIBE|1.6659379660953851E7|
|      SUR|1.3142652615603926E7|
|   BOGOTA|1.4238203487278929E7|
|   CENTRO|1.5640429774039075E7|
+---------+--------------------+



In [293]:
df_region_moda=df_spark.groupBy('region').count().alias('counts')
df_region_moda.show()

+---------+-----+
|   region|count|
+---------+-----+
|ANTIOQUIA|17498|
|   CARIBE|12709|
|      SUR|15348|
|   BOGOTA|28017|
|   CENTRO|15675|
+---------+-----+



In [294]:
df_ocuacion_deuda= df_spark.groupBy('ocupacion').avg('endeudamiento').show()

+--------------------+--------------------+
|           ocupacion|  avg(endeudamiento)|
+--------------------+--------------------+
|EMPLEADO SECTOR P...|1.2410469460981164E7|
|             TAXISTA|  2.04866593046875E7|
|                null|1.3825151085135136E7|
|NO APLICA A ESTE ...| 1.770801854432159E7|
|         AMA DE CASA| 1.020864353488372E7|
|          SIN OFICIO|1.2819059342742719E7|
|  TRABAJADOR OFICIAL|1.2332329344594594E7|
|EMPLEADO SECTOR P...|1.2243880926898362E7|
|EMPLEADO BANCOLOMBIA|           6437636.5|
|       INDEPENDIENTE|2.4730633660613343E7|
|     EMPLEADO FILIAL| 1.075199542857143E7|
|            JUBILADO| 1.622143445371367E7|
+--------------------+--------------------+



In [296]:
df_cuadrante_deuda= df_spark.groupBy('cuadrante').avg('endeudamiento').show()

+--------------------+--------------------+
|           cuadrante|  avg(endeudamiento)|
+--------------------+--------------------+
|TIENE VOLUNTAD Y ...| 1.719154621889488E7|
|TIENE VOLUNTAD Y ...| 1.489277308024154E7|
|                null|1.0842978152859375E7|
|NO TIENE VOLUNTAD...| 1.226303705357143E7|
|       SIN CUADRANTE|   8642141.573870536|
|TIENE VOLUNTAD Y ...| 2.109910006039823E7|
|NO TIENE VOLUNTAD...|1.6026260628087219E7|
+--------------------+--------------------+

